In [1]:
# For Google Colab use
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd '/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder'    
except ModuleNotFoundError:
    pass

In [2]:
# Imports
from __future__ import division

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from numpy.random import seed
from time import time
from pathlib import Path

import tensorflow as tf

import keras
from keras import preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import *
from keras.applications import *
from keras import metrics
from keras.losses import binary_crossentropy
from keras import backend as K

try:
    import face_recognition
except ModuleNotFoundError:
    !pip install face_recognition

from Util import pipeline
from Baseline.classifiers import *
from Util import experiment_tuner_models

In [3]:
def evaluate_model(TEST_MODEL, EXPERIMENT_NAME:str, TEST_DIR:str, BACTH_SIZE:int=64, \
                   IMG_HEIGHT:int=256, IMG_WIDTH:int=256, DATA_GENERATOR_SEED:int=1337, \
                   HISTORY=None):
    
    from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
    
    EXPERIMENT_NAME = EXPERIMENT_NAME.replace(" ", "_")
    EXPERIMENT_FOLDER_NAME = f'./Results/{EXPERIMENT_NAME}'
    Path(EXPERIMENT_FOLDER_NAME).mkdir(parents=True, exist_ok=True)

    # Define testing datagen
    print('Defining testing datagen...')
    TEST_DATAGEN = ImageDataGenerator(rescale=1./255)
    TEST_GENERATOR = TEST_DATAGEN.flow_from_directory(directory = TEST_DIR,
                                                      batch_size = BACTH_SIZE,
                                                      class_mode = 'binary', 
                                                      target_size =  (IMG_HEIGHT, IMG_WIDTH),                                
                                                      seed = DATA_GENERATOR_SEED,
                                                      follow_links = True)

    # Evaluate test set and get metrics, e.g. loss
    print('Evaluating test set and getting metrics...')
    TEST_HISORY = TEST_MODEL.evaluate(TEST_GENERATOR,
                                      batch_size = BACTH_SIZE,
                                      return_dict = True)
    TEST_GENERATOR.reset()

    # Get predictions and ground truths
    print('Predicting on test set...')
    Y_pred = (TEST_MODEL.predict(TEST_GENERATOR)).ravel()
    Y_true = (TEST_GENERATOR.classes).ravel()

    # Plot AUC and save plot
    print('Plotting AUC...')
    fpr, tpr, _ = roc_curve(Y_true, Y_pred)
    try:
        roc_auc = auc(fpr, tpr)
    except:
        roc_auc = None
        roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='magenta', lw=2, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.01])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{EXPERIMENT_NAME.replace("_", " ")} ROC')
    plt.legend(loc="lower right")
    print(f'Saving AUC to {EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_AUC.pdf')
    plt.savefig(f'{EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_AUC.pdf')

    # Create file, write metrics
    print(f'Saving summary file to {EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_summary.txt')
    summary_file = open(f'{EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_summary.txt', 'w')
    summary_file.write(str(TEST_HISORY).replace(', \'', ':\n\n'))
    print(str(TEST_HISORY).replace(', \'', ':\n'))

    # find classification thershold which results in max auc
    roc_aucs = []
    for thrshld in map(lambda x: x/1000.0, range(0, 1000+1)):
        fpr, tpr, _ = roc_curve(Y_true, (Y_pred >= thrshld).astype(int))
        roc_aucs.append((auc(fpr, tpr), thrshld))
    max_auc_thrshld = max(roc_aucs)[1]

    # use max auc classification thershold to get confusion matrix and classification report
    y_pred = (Y_pred >= max_auc_thrshld).astype(int)
    summary_output_text = f'''\n\nTHRESHOLD = {max_auc_thrshld}
    \nCONFUSION MATRIX\n{confusion_matrix(Y_true, y_pred)}
    \nCLASSIFICATION REPORT\n{classification_report(Y_true, y_pred, target_names = ["REAL", "FAKE"])}
    _____________________________________________________\n'''
    summary_file.write(summary_output_text)
    summary_file.write(f'''\n{list(zip(Y_true, Y_pred))}''')
    print(summary_output_text)
    summary_file.close()

    # plot history (of training), if provided
    if HISTORY is not None:
        print('Plotting training history x4 plots...')
        acc = HISTORY.history['acc']
        auc = HISTORY.history['auc']
        loss = HISTORY.history['loss']
        fp = HISTORY.history['fp']

        val_acc = HISTORY.history['val_acc']
        val_auc = HISTORY.history['val_auc']
        val_loss = HISTORY.history['val_loss']
        val_fp = HISTORY.history['val_fp']

        epochs = range(len(acc))
        fig, axs = plt.subplots(2, 2, figsize=(10,10))
        fig.suptitle(f'{EXPERIMENT_NAME.replace("_", " ")} Train & Validation Metrics')

        axs[0, 0].plot(epochs, acc, 'r', label='Train Binary Accuracy')
        axs[0, 0].plot(epochs, val_acc, 'b', label='Validation Binary Accuracy')
        axs[0, 0].set_title('Binary Accuracy')
        axs[0, 0].set_ylabel('Accuracy')
        axs[0, 0].set_xlabel('Epochs')
        axs[0, 0].legend()

        axs[0, 1].plot(epochs, loss, 'r', label='Train Loss')
        axs[0, 1].plot(epochs, val_loss, 'b', label='Validation Loss')
        axs[0, 1].set_title('Loss')
        axs[0, 1].set_ylabel('Loss')
        axs[0, 1].set_xlabel('Epochs')
        axs[0, 1].legend()

        axs[1, 0].plot(epochs, auc, 'r', label='Train AUC')
        axs[1, 0].plot(epochs, val_auc, 'b', label='Validation AUC')
        axs[1, 0].set_title('AUC')
        axs[1, 0].set_ylabel('AUC')
        axs[1, 0].set_xlabel('Epochs')
        axs[1, 0].legend()

        axs[1, 1].plot(epochs, fp, 'r', label='Train False Positives')
        axs[1, 1].plot(epochs, val_fp, 'b', label='Validation False Positives')
        axs[1, 1].set_title('False Positives')
        axs[1, 1].set_ylabel('FP')
        axs[1, 1].set_xlabel('Epochs')
        axs[1, 1].legend()
        
        print(f'Saving history plots to {EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_history.pdf')
        fig.savefig(f'{EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_history.pdf')
    
    return Y_true, Y_pred

In [ ]:
EXPERIMENT_NAME = 'Feature_Extractor_MesoInception4_on_{df_name}_Dataset'

acc = HISTORY.history['acc']
auc = HISTORY.history['auc']
loss = HISTORY.history['loss']
fp = HISTORY.history['fp']

val_acc = HISTORY.history['val_acc']
val_auc = HISTORY.history['val_auc']
val_loss = HISTORY.history['val_loss']
val_fp = HISTORY.history['val_fp']

epochs = range(len(acc))
fig, axs = plt.subplots(2, 2, figsize=(10,10))
fig.suptitle(f'{EXPERIMENT_NAME.replace("_", " ")} Train & Validation Metrics')

axs[0, 0].plot(epochs, acc, 'r', label='Train Binary Accuracy')
axs[0, 0].plot(epochs, val_acc, 'b', label='Validation Binary Accuracy')
axs[0, 0].set_title('Binary Accuracy')
axs[0, 0].set_ylabel('Accuracy')
axs[0, 0].set_xlabel('Epochs')
axs[0, 0].legend()

axs[0, 1].plot(epochs, loss, 'r', label='Train Loss')
axs[0, 1].plot(epochs, val_loss, 'b', label='Validation Loss')
axs[0, 1].set_title('Loss')
axs[0, 1].set_ylabel('Loss')
axs[0, 1].set_xlabel('Epochs')
axs[0, 1].legend()

axs[1, 0].plot(epochs, auc, 'r', label='Train AUC')
axs[1, 0].plot(epochs, val_auc, 'b', label='Validation AUC')
axs[1, 0].set_title('AUC')
axs[1, 0].set_ylabel('AUC')
axs[1, 0].set_xlabel('Epochs')
axs[1, 0].legend()

axs[1, 1].plot(epochs, fp, 'r', label='Train False Positives')
axs[1, 1].plot(epochs, val_fp, 'b', label='Validation False Positives')
axs[1, 1].set_title('False Positives')
axs[1, 1].set_ylabel('FP')
axs[1, 1].set_xlabel('Epochs')
axs[1, 1].legend()

print(f'Saving history plots to {EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_history.pdf')
fig.savefig(f'{EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_history.pdf')

In [4]:
# for df_name, df_type in tqdm([('Random_Frame', 'rnd'), ('Frame_Averaging', 'avg'), ('Frame_Difference', 'diff')]):
#         for weight_path in [f'./Baseline/weights/MesoInception_F2F.h5', f'./Baseline/weights/MesoInception_DF.h5']:
#             weight_name = weight_path.split('_')[-1].replace('.h5', '')
#             print(df_type.upper(), weight_name.upper())

#             TEST_MODEL = None
#             TEST_MODEL = MesoInception4().model
#             TEST_MODEL.load_weights(weight_path)

#             evaluate_model(TEST_MODEL = TEST_MODEL,
#                            EXPERIMENT_NAME = f'Untrained_MesoInception4_({weight_name})_on_{df_name}_Dataset',
#                            TEST_DIR = f'./Celeb-DF-v2/Celeb-{df_type}-30-test')

In [ ]:
# General model settings
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 256, 256, 3
EPOCHS = 100
DATA_GENERATOR_SEED = 1337
BATCH_SIZE = 64
VALIDATION_SPLIT = 0.2

tf.random.set_seed(DATA_GENERATOR_SEED)
seed(DATA_GENERATOR_SEED)

# Pick dataset; DF_TYPE={'rnd', 'avg'}
for df_name, DF_TYPE in tqdm([('Random_Frame', 'rnd'), ('Frame_Averaging', 'avg'), ('Frame_Difference', 'diff')][::-1]):
    TRAIN_VAL_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}-30'
    print(f'{DF_TYPE}!!!')

    print('Setting up datagens...')
    TRAIN_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0, horizontal_flip = True, fill_mode='nearest', validation_split = VALIDATION_SPLIT)
    TRAIN_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                        batch_size = BATCH_SIZE,
                                                        class_mode = 'binary', 
                                                        target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                        subset = 'training',
                                                        seed = DATA_GENERATOR_SEED,
                                                        follow_links = True)

    VAL_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0, validation_split = VALIDATION_SPLIT)
    VALIDATION_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                             batch_size = BATCH_SIZE,
                                                             class_mode = 'binary', 
                                                             target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                             subset = 'validation',
                                                             seed = DATA_GENERATOR_SEED,
                                                             follow_links = True)

    # Train MesoInception4 w/ F2F weights
    print('Loading MesoInception4 w/ F2F weights...')
    TEST_MODEL = MesoInception4().model
    TEST_MODEL.compile(optimizer = TEST_MODEL.optimizer,
                       loss = TEST_MODEL.loss,
                       metrics = TEST_MODEL.metrics + [metrics.BinaryAccuracy(name = 'acc'),
                                                       metrics.AUC(name = 'auc'),
                                                       metrics.FalsePositives(name = 'fp')])
    
    WEIGHTS_PATH = './Baseline/weights/MesoInception_F2F.h5'
    TEST_MODEL.load_weights(WEIGHTS_PATH) 

    # Make last layer trainable
    for layer in TEST_MODEL.layers:
        layer.trainable = False
    TEST_MODEL.layers[-1].trainable = True

    train_gen_list = list(TRAIN_GENERATOR.classes)
    val_gen_list = list(VALIDATION_GENERATOR.classes)

    train_neg, train_pos = train_gen_list.count(0), train_gen_list.count(1)
    val_neg, val_pos = val_gen_list.count(0), val_gen_list.count(1)

    pos = train_pos + val_pos
    neg = train_neg + val_neg
    total = pos + neg

    weight_for_0 = (1.0 / neg)*(total)/2.0 
    weight_for_1 = (1.0 / pos)*(total)/2.0

    CLASS_WEIGHT = {0: weight_for_0, 1: weight_for_1}
    print(f'Class weights = {CLASS_WEIGHT}')

    EARLY_STOP = EarlyStopping(monitor='val_auc',
                            patience=EPOCHS//10,
                            mode='max',
                            verbose=1,
                            restore_best_weights=True)
    
    MODEL_CHECKPOINT = ModelCheckpoint(filepath = f'./Weights/FE-{DF_TYPE.upper()}', 
                                       monitor='val_auc', 
                                       verbose=0, 
                                       save_best_only=True,
                                       save_weights_only=True,
                                       mode='max', 
                                       save_freq='epoch')
    
    print(f'Early stopping in {EARLY_STOP.patience}')

    HISTORY = TEST_MODEL.fit(TRAIN_GENERATOR,
                            steps_per_epoch = TRAIN_GENERATOR.n//TRAIN_GENERATOR.batch_size,
                            validation_data = VALIDATION_GENERATOR,
                            validation_steps = VALIDATION_GENERATOR.n//VALIDATION_GENERATOR.batch_size,
                            epochs = EPOCHS,
                            verbose = 1,
                            class_weight = CLASS_WEIGHT,
                            callbacks = [EARLY_STOP, MODEL_CHECKPOINT])
    

#     weight_name = WEIGHTS_PATH.split('_')[-1].replace('.h5', '')
    evaluate_model(TEST_MODEL = TEST_MODEL,
                   EXPERIMENT_NAME = f'Feature_Extractor_MesoInception4_on_{df_name}_Dataset',
                   TEST_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}-30-test',
                   HISTORY = HISTORY)

  0%|          | 0/3 [00:00<?, ?it/s]

diff!!!
Setting up datagens...
Found 44393 images belonging to 2 classes.
Found 11097 images belonging to 2 classes.
Loading MesoInception4 w/ F2F weights...
Class weights = {0: 4.930691309756531, 1: 0.5564246034133526}
Early stopping in 10
Epoch 1/100


120/693 [====>.........................] - ETA: 0s - loss: 0.5752 - acc: 0.6094 - auc: 0.3919 - fp: 8.00 - ETA: 1:28 - loss: 0.4379 - acc: 0.6875 - auc: 0.3542 - fp: 11.000 - ETA: 1:57 - loss: 0.4545 - acc: 0.7240 - auc: 0.4488 - fp: 17.000 - ETA: 4:43 - loss: 0.4402 - acc: 0.7461 - auc: 0.4670 - fp: 22.000 - ETA: 5:55 - loss: 0.4570 - acc: 0.7500 - auc: 0.4493 - fp: 28.000 - ETA: 6:45 - loss: 0.4708 - acc: 0.7604 - auc: 0.4373 - fp: 35.000 - ETA: 7:11 - loss: 0.4454 - acc: 0.7768 - auc: 0.4347 - fp: 39.000 - ETA: 7:42 - loss: 0.4796 - acc: 0.7734 - auc: 0.4426 - fp: 49.000 - ETA: 8:01 - loss: 0.4681 - acc: 0.7795 - auc: 0.4585 - fp: 54.000 - ETA: 8:18 - loss: 0.4647 - acc: 0.7828 - auc: 0.4535 - fp: 60.000 - ETA: 7:48 - loss: 0.4686 - acc: 0.7855 - auc: 0.4538 - fp: 67.000 - ETA: 7:50 - loss: 0.4794 - acc: 0.7826 - auc: 0.4399 - fp: 74.000 - ETA: 8:03 - loss: 0.4580 - acc: 0.7909 - auc: 0.4438 - fp: 76.000 - ETA: 7:40 - loss: 0.4599 - acc: 0.7891 - auc: 0.4623 - fp: 82.000 - ETA: 7:20

477/693 [===================>..........] - ETA: 4:05 - loss: 0.2647 - acc: 0.5344 - auc: 0.5618 - fp: 998.00 - ETA: 4:04 - loss: 0.2647 - acc: 0.5345 - auc: 0.5613 - fp: 1003.000 - ETA: 4:03 - loss: 0.2645 - acc: 0.5344 - auc: 0.5613 - fp: 1004.000 - ETA: 4:03 - loss: 0.2647 - acc: 0.5346 - auc: 0.5618 - fp: 1008.000 - ETA: 4:03 - loss: 0.2649 - acc: 0.5349 - auc: 0.5617 - fp: 1013.000 - ETA: 4:02 - loss: 0.2648 - acc: 0.5351 - auc: 0.5619 - fp: 1015.000 - ETA: 4:01 - loss: 0.2648 - acc: 0.5351 - auc: 0.5621 - fp: 1017.000 - ETA: 4:01 - loss: 0.2646 - acc: 0.5353 - auc: 0.5622 - fp: 1019.000 - ETA: 4:00 - loss: 0.2646 - acc: 0.5353 - auc: 0.5622 - fp: 1023.000 - ETA: 4:00 - loss: 0.2645 - acc: 0.5355 - auc: 0.5626 - fp: 1024.000 - ETA: 3:59 - loss: 0.2644 - acc: 0.5354 - auc: 0.5630 - fp: 1027.000 - ETA: 3:59 - loss: 0.2643 - acc: 0.5354 - auc: 0.5630 - fp: 1029.000 - ETA: 3:57 - loss: 0.2643 - acc: 0.5354 - auc: 0.5631 - fp: 1032.000 - ETA: 3:57 - loss: 0.2645 - acc: 0.5353 - auc: 0.5

594/693 [========================>.....] - ETA: 2:41 - loss: 0.2591 - acc: 0.5314 - auc: 0.5722 - fp: 1268.000 - ETA: 2:40 - loss: 0.2589 - acc: 0.5314 - auc: 0.5723 - fp: 1269.000 - ETA: 2:39 - loss: 0.2588 - acc: 0.5314 - auc: 0.5722 - fp: 1271.000 - ETA: 2:38 - loss: 0.2590 - acc: 0.5316 - auc: 0.5724 - fp: 1276.000 - ETA: 2:38 - loss: 0.2589 - acc: 0.5315 - auc: 0.5725 - fp: 1278.000 - ETA: 2:37 - loss: 0.2589 - acc: 0.5316 - auc: 0.5726 - fp: 1279.000 - ETA: 2:36 - loss: 0.2587 - acc: 0.5317 - auc: 0.5730 - fp: 1280.000 - ETA: 2:35 - loss: 0.2588 - acc: 0.5317 - auc: 0.5727 - fp: 1285.000 - ETA: 2:35 - loss: 0.2587 - acc: 0.5315 - auc: 0.5729 - fp: 1286.000 - ETA: 2:34 - loss: 0.2587 - acc: 0.5314 - auc: 0.5730 - fp: 1290.000 - ETA: 2:33 - loss: 0.2589 - acc: 0.5316 - auc: 0.5732 - fp: 1293.000 - ETA: 2:32 - loss: 0.2588 - acc: 0.5318 - auc: 0.5734 - fp: 1295.000 - ETA: 2:32 - loss: 0.2586 - acc: 0.5320 - auc: 0.5735 - fp: 1295.000 - ETA: 2:31 - loss: 0.2586 - acc: 0.5318 - auc: 0

...

121/693 [====>.........................] - ETA: 0s - loss: 0.1377 - acc: 0.7500 - auc: 0.5927 - fp: 2.00 - ETA: 1:27 - loss: 0.1501 - acc: 0.7656 - auc: 0.8531 - fp: 4.00 - ETA: 1:57 - loss: 0.1709 - acc: 0.7552 - auc: 0.8188 - fp: 5.00 - ETA: 2:11 - loss: 0.1709 - acc: 0.7656 - auc: 0.8351 - fp: 7.00 - ETA: 2:20 - loss: 0.1678 - acc: 0.7688 - auc: 0.8141 - fp: 9.00 - ETA: 2:26 - loss: 0.1782 - acc: 0.7604 - auc: 0.7929 - fp: 11.000 - ETA: 2:30 - loss: 0.1800 - acc: 0.7545 - auc: 0.7849 - fp: 14.000 - ETA: 2:33 - loss: 0.1803 - acc: 0.7422 - auc: 0.7906 - fp: 16.000 - ETA: 2:35 - loss: 0.1735 - acc: 0.7413 - auc: 0.7954 - fp: 16.000 - ETA: 2:37 - loss: 0.1715 - acc: 0.7500 - auc: 0.8010 - fp: 18.000 - ETA: 2:38 - loss: 0.1632 - acc: 0.7571 - auc: 0.8183 - fp: 18.000 - ETA: 2:39 - loss: 0.1643 - acc: 0.7604 - auc: 0.8166 - fp: 21.000 - ETA: 2:40 - loss: 0.1619 - acc: 0.7572 - auc: 0.8133 - fp: 22.000 - ETA: 2:41 - loss: 0.1666 - acc: 0.7556 - auc: 0.8049 - fp: 27.000 - ETA: 2:41 - loss:

481/693 [===================>..........] - ETA: 1:24 - loss: 0.1809 - acc: 0.7267 - auc: 0.8069 - fp: 621.00 - ETA: 1:24 - loss: 0.1808 - acc: 0.7268 - auc: 0.8071 - fp: 622.00 - ETA: 1:23 - loss: 0.1808 - acc: 0.7266 - auc: 0.8074 - fp: 623.00 - ETA: 1:23 - loss: 0.1811 - acc: 0.7261 - auc: 0.8070 - fp: 626.00 - ETA: 1:23 - loss: 0.1812 - acc: 0.7258 - auc: 0.8069 - fp: 627.00 - ETA: 1:23 - loss: 0.1811 - acc: 0.7254 - auc: 0.8067 - fp: 627.00 - ETA: 1:22 - loss: 0.1814 - acc: 0.7250 - auc: 0.8060 - fp: 631.00 - ETA: 1:22 - loss: 0.1814 - acc: 0.7244 - auc: 0.8058 - fp: 631.00 - ETA: 1:22 - loss: 0.1815 - acc: 0.7242 - auc: 0.8056 - fp: 632.00 - ETA: 1:22 - loss: 0.1815 - acc: 0.7244 - auc: 0.8057 - fp: 633.00 - ETA: 1:21 - loss: 0.1817 - acc: 0.7241 - auc: 0.8055 - fp: 634.00 - ETA: 1:21 - loss: 0.1819 - acc: 0.7237 - auc: 0.8050 - fp: 636.00 - ETA: 1:21 - loss: 0.1818 - acc: 0.7235 - auc: 0.8051 - fp: 637.00 - ETA: 1:21 - loss: 0.1817 - acc: 0.7232 - auc: 0.8050 - fp: 637.00 - ETA: 

601/693 [=========================>....] - ETA: 53s - loss: 0.1822 - acc: 0.7231 - auc: 0.8043 - fp: 816.000 - ETA: 53s - loss: 0.1823 - acc: 0.7232 - auc: 0.8042 - fp: 819.000 - ETA: 53s - loss: 0.1824 - acc: 0.7233 - auc: 0.8039 - fp: 822.000 - ETA: 52s - loss: 0.1822 - acc: 0.7235 - auc: 0.8041 - fp: 822.000 - ETA: 52s - loss: 0.1823 - acc: 0.7234 - auc: 0.8040 - fp: 827.000 - ETA: 52s - loss: 0.1825 - acc: 0.7236 - auc: 0.8037 - fp: 832.000 - ETA: 52s - loss: 0.1826 - acc: 0.7236 - auc: 0.8035 - fp: 834.000 - ETA: 51s - loss: 0.1825 - acc: 0.7236 - auc: 0.8037 - fp: 835.000 - ETA: 51s - loss: 0.1825 - acc: 0.7236 - auc: 0.8037 - fp: 836.000 - ETA: 51s - loss: 0.1825 - acc: 0.7237 - auc: 0.8038 - fp: 839.000 - ETA: 51s - loss: 0.1825 - acc: 0.7235 - auc: 0.8036 - fp: 841.000 - ETA: 50s - loss: 0.1825 - acc: 0.7236 - auc: 0.8037 - fp: 842.000 - ETA: 50s - loss: 0.1826 - acc: 0.7235 - auc: 0.8035 - fp: 845.000 - ETA: 50s - loss: 0.1826 - acc: 0.7234 - auc: 0.8036 - fp: 847.000 - ETA: 

693/693 [==============================] - ETA: 23s - loss: 0.1812 - acc: 0.7269 - auc: 0.8053 - fp: 1033.00 - ETA: 22s - loss: 0.1812 - acc: 0.7270 - auc: 0.8053 - fp: 1036.00 - ETA: 22s - loss: 0.1811 - acc: 0.7271 - auc: 0.8054 - fp: 1037.00 - ETA: 22s - loss: 0.1810 - acc: 0.7272 - auc: 0.8055 - fp: 1038.00 - ETA: 22s - loss: 0.1810 - acc: 0.7274 - auc: 0.8057 - fp: 1041.00 - ETA: 21s - loss: 0.1813 - acc: 0.7274 - auc: 0.8053 - fp: 1046.00 - ETA: 21s - loss: 0.1813 - acc: 0.7276 - auc: 0.8054 - fp: 1047.00 - ETA: 21s - loss: 0.1812 - acc: 0.7278 - auc: 0.8054 - fp: 1049.00 - ETA: 21s - loss: 0.1812 - acc: 0.7279 - auc: 0.8056 - fp: 1050.00 - ETA: 20s - loss: 0.1812 - acc: 0.7280 - auc: 0.8056 - fp: 1052.00 - ETA: 20s - loss: 0.1812 - acc: 0.7281 - auc: 0.8055 - fp: 1055.00 - ETA: 20s - loss: 0.1811 - acc: 0.7283 - auc: 0.8058 - fp: 1056.00 - ETA: 20s - loss: 0.1811 - acc: 0.7283 - auc: 0.8058 - fp: 1058.00 - ETA: 19s - loss: 0.1810 - acc: 0.7283 - auc: 0.8059 - fp: 1060.00 - ETA: 

120/693 [====>.........................] - ETA: 0s - loss: 0.1236 - acc: 0.7344 - auc: 0.9464 - fp: 0.0000e+ - ETA: 1:28 - loss: 0.2124 - acc: 0.6797 - auc: 0.8058 - fp: 3.0000 - ETA: 1:57 - loss: 0.2425 - acc: 0.6406 - auc: 0.7532 - fp: 8.00 - ETA: 2:12 - loss: 0.2208 - acc: 0.6680 - auc: 0.7801 - fp: 9.00 - ETA: 2:21 - loss: 0.2063 - acc: 0.6562 - auc: 0.7900 - fp: 10.000 - ETA: 2:27 - loss: 0.2009 - acc: 0.6589 - auc: 0.7861 - fp: 12.000 - ETA: 2:31 - loss: 0.1971 - acc: 0.6652 - auc: 0.7860 - fp: 13.000 - ETA: 2:34 - loss: 0.1967 - acc: 0.6699 - auc: 0.7834 - fp: 14.000 - ETA: 2:37 - loss: 0.1919 - acc: 0.6753 - auc: 0.7912 - fp: 14.000 - ETA: 2:38 - loss: 0.1909 - acc: 0.6766 - auc: 0.7917 - fp: 15.000 - ETA: 2:40 - loss: 0.1823 - acc: 0.6832 - auc: 0.8032 - fp: 15.000 - ETA: 2:41 - loss: 0.1815 - acc: 0.6849 - auc: 0.8015 - fp: 17.000 - ETA: 2:42 - loss: 0.1799 - acc: 0.6947 - auc: 0.8044 - fp: 18.000 - ETA: 2:44 - loss: 0.1778 - acc: 0.7031 - auc: 0.8064 - fp: 20.000 - ETA: 2:45

In [ ]:
# # General model settings
# IMG_HEIGHT, IMG_WIDTH = 71, 71
# EPOCHS = 5
# DATA_GENERATOR_SEED = 1337
# BATCH_SIZE = 64
# VALIDATION_SPLIT = 0.2

# tf.random.set_seed(DATA_GENERATOR_SEED)
# seed(DATA_GENERATOR_SEED)

# for EXPERIMENT_TYPE in tqdm(['optimiser', 'structure']):
#     for df_name, DF_TYPE in tqdm([('Random_Frame', 'rnd'), ('Frame_Averaging', 'avg'), ('Frame_Difference', 'diff')]):
        
#         EXPERIMENT_NAME = f'Hyperband({EXPERIMENT_TYPE.title()})_{df_name}_Dataset'
#         EXPERIMENT_FOLDER_NAME = f'./Results/{EXPERIMENT_NAME}'
#         Path(EXPERIMENT_FOLDER_NAME).mkdir(parents=True, exist_ok=True)
#         print(EXPERIMENT_NAME)

#         TRAIN_VAL_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}-30'

#         print('Defining train and validation datagens...')
#         TRAIN_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0, horizontal_flip = True, fill_mode='nearest', validation_split = VALIDATION_SPLIT)
#         TRAIN_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
#                                                             batch_size = BATCH_SIZE,
#                                                             class_mode = 'binary', 
#                                                             target_size = (IMG_HEIGHT, IMG_WIDTH),
#                                                             subset = 'training',
#                                                             seed = DATA_GENERATOR_SEED,
#                                                             follow_links = True)

#         VAL_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0, validation_split = VALIDATION_SPLIT)
#         VALIDATION_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
#                                                                  batch_size = BATCH_SIZE,
#                                                                  class_mode = 'binary', 
#                                                                  target_size = (IMG_HEIGHT, IMG_WIDTH),
#                                                                  subset = 'validation',
#                                                                  seed = DATA_GENERATOR_SEED,
#                                                                  follow_links = True)
        
#         print('Calculating class weights')
#         train_gen_list = list(TRAIN_GENERATOR.classes)
#         val_gen_list = list(VALIDATION_GENERATOR.classes)

#         train_neg, train_pos = train_gen_list.count(0), train_gen_list.count(1)
#         val_neg, val_pos = val_gen_list.count(0), val_gen_list.count(1)

#         pos = train_pos + val_pos
#         neg = train_neg + val_neg
#         total = pos + neg

#         weight_for_0 = (1.0 / neg) * (total) / 2.0 
#         weight_for_1 = (1.0 / pos) * (total) / 2.0

#         CLASS_WEIGHT = {0: weight_for_0, 1: weight_for_1}
#         print(f'Class weights = {CLASS_WEIGHT}')

#         print(f'Setting up Hyperband with {EXPERIMENT_TYPE}')
#         tuner = Hyperband(eval(f'experiment_tuner_models.{EXPERIMENT_TYPE}_experiment_model'),
#                           objective = Objective('val_auc', direction = 'max'),
#                           max_epochs = EPOCHS,
#                           directory = './Results/',
#                           project_name = f'{EXPERIMENT_NAME}/',
#                           overwrite = True)

#         tuner.search_space_summary()
#         tuner.search(TRAIN_GENERATOR,
#                      validation_data = VALIDATION_GENERATOR,
#                      epochs = EPOCHS,
#                      steps_per_epoch = TRAIN_GENERATOR.n // TRAIN_GENERATOR.batch_size,
#                      validation_steps = VALIDATION_GENERATOR.n // VALIDATION_GENERATOR.batch_size,
#                      class_weight = CLASS_WEIGHT,
#                      verbose = 1,
#                      callbacks = [EarlyStopping(monitor = 'val_auc',
#                                                 patience = EPOCHS//5+1,
#                                                 mode = 'max',
#                                                 verbose = 1)])
#         print(tuner.results_summary())
#         print(tuner.get_best_models())

#         print(f'Saving tuner summary file to {EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_tuner.txt')
#         tuner_file = open(f'{EXPERIMENT_FOLDER_NAME}/{EXPERIMENT_NAME}_tuner.txt', 'w')
#         tuner_file.write(tuner.results_summary())
#         tuner_file.write(tuner.get_best_models())
#     #     summary_file.write(str(tuner.results_summary()).replace(', \'', ':\n\n'))
#     #     summary_file.write(str(tuner.get_best_models()).replace(', \'', ':\n\n'))
#         tuner_file.close()

#         evaluate_model(TEST_MODEL = tuner.get_best_models()[0],
#                        EXPERIMENT_NAME = EXPERIMENT_NAME,
#                        TEST_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}-30-test',
#                        HISTORY = tuner.get_best_models()[0].history)